# Preprocessing

In [1]:
import re
import pandas as pd
import tensorflow as tf
import numpy as np
import unicodedata

In [2]:
num_samples = 70000

In [3]:
def to_ascii(s):
    return ''.join(c for c in unicodedata.normalize('NFD', s) if unicodedata.category(c) != 'Mn')

def preprocess_sentence(sent) :
    sent = to_ascii(sent.lower())
    sent = re.sub(r"([?.!,¿])", r" \1", sent)
    sent = re.sub(r"[^a-zA-Z!.?,]+", r" ", sent)
    sent = re.sub(r"\s+", " ", sent)
    
    return sent

In [4]:
preprocess_sentence("Aab,?d ㄷ   test.")

'aab , ?d test .'

In [5]:
def load_preprocessed_data() :
    encoder_input, decoder_input, decoder_target = [], [], []
    
    with open("DataSet/spa.txt","r") as lines :
        for i, line in enumerate(lines) :
            src_line, tar_line, _ = line.strip().split('\t')
            src_line = [w for w in preprocess_sentence(src_line).split()]
            
            tar_line = preprocess_sentence(tar_line)
            tar_line_in = [w for w in ("<sos> " + tar_line).split()]
            tar_line_out = [w for w in (tar_line + " <eos>").split()]
            
            encoder_input.append(src_line)
            decoder_input.append(tar_line_in)
            decoder_target.append(tar_line_out)
            
            if i==num_samples-1 :
                break
                
    return encoder_input, decoder_input, decoder_target

In [6]:
sents_en_in, sents_spa_in, sents_spa_out = load_preprocessed_data()

In [7]:
print(sents_en_in[:7])
print(sents_spa_in[:7])
print(sents_spa_out[:7])

[['go', '.'], ['go', '.'], ['go', '.'], ['go', '.'], ['hi', '.'], ['hi', '.'], ['run', '!']]
[['<sos>', 've', '.'], ['<sos>', 'vete', '.'], ['<sos>', 'vaya', '.'], ['<sos>', 'vayase', '.'], ['<sos>', 'hola', '.'], ['<sos>', 'hola'], ['<sos>', 'corre', '!']]
[['ve', '.', '<eos>'], ['vete', '.', '<eos>'], ['vaya', '.', '<eos>'], ['vayase', '.', '<eos>'], ['hola', '.', '<eos>'], ['hola', '<eos>'], ['corre', '!', '<eos>']]


In [8]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer

In [9]:
tokenizer_en = Tokenizer(filters="", lower=False)
tokenizer_en.fit_on_texts(sents_en_in)
encoder_input = tokenizer_en.texts_to_sequences(sents_en_in)
encoder_input = pad_sequences(encoder_input, padding="post")

tokenizer_spa = Tokenizer(filters="", lower=False)
tokenizer_spa.fit_on_texts(sents_spa_in)
tokenizer_spa.fit_on_texts(sents_spa_out)

decoder_input = tokenizer_spa.texts_to_sequences(sents_spa_in)
decoder_input = pad_sequences(decoder_input, padding="post")
decoder_target = tokenizer_spa.texts_to_sequences(sents_spa_out)
decoder_target = pad_sequences(decoder_target, padding="post")

In [10]:
print(encoder_input.shape)
print(decoder_input.shape)
print(decoder_target.shape)

(70000, 11)
(70000, 18)
(70000, 18)


In [11]:
src_vocab_size = len(tokenizer_en.word_index)+1
tar_vocab_size = len(tokenizer_spa.word_index)+1
print("Size of english voca set:",src_vocab_size)
print("Size of spanish voca set:",tar_vocab_size)

Size of english voca set: 8110
Size of spanish voca set: 15842


In [12]:
src_to_index = tokenizer_en.word_index
index_to_src = tokenizer_en.index_word
tar_to_index = tokenizer_spa.word_index
index_to_tar = tokenizer_spa.index_word

In [13]:
indices = np.arange(encoder_input.shape[0])
np.random.shuffle(indices)
print("random sequence :",indices)

random sequence : [69432 25048 21868 ... 57805 11209 12168]


In [14]:
encoder_input = encoder_input[indices]
decoder_input = decoder_input[indices]
decoder_target = decoder_target[indices]

In [15]:
print(encoder_input[0])
print(decoder_input[0])
print(decoder_target[0])

[ 19 149   8 662   9 925   1   0   0   0   0]
[  2  21  55  15 287  11  12 948   1   0   0   0   0   0   0   0   0   0]
[ 21  55  15 287  11  12 948   1   3   0   0   0   0   0   0   0   0   0]


In [16]:
n_of_val = int(num_samples*0.1)
print("Number of test data : ",n_of_val)

Number of test data :  7000


In [17]:
encoder_input_train = encoder_input[:-n_of_val]
decoder_input_train = decoder_input[:-n_of_val]
decoder_target_train = decoder_target[:-n_of_val]

encoder_input_test = encoder_input[-n_of_val:]
decoder_input_test = decoder_input[-n_of_val:]
decoder_target_test = decoder_target[-n_of_val:]

In [18]:
print(encoder_input_train.shape)
print(encoder_input_test.shape)

(63000, 11)
(7000, 11)


# Training Model

In [19]:
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense, Masking
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

In [20]:
embedding_dim = 512
hidden_units = 512

In [21]:
#Encoder
encoder_inputs = Input(shape=(None,))
enc_emb = Embedding(src_vocab_size, embedding_dim)(encoder_inputs)
enc_masking = Masking(mask_value=0)(enc_emb)
encoder_lstm = LSTM(hidden_units, return_state=True)
encoder_outputs, state_h, state_c = encoder_lstm(enc_masking)
encoder_states = [state_h, state_c]

In [22]:
#Decoder
decoder_inputs = Input(shape=(None,))
dec_emb_layer = Embedding(tar_vocab_size, hidden_units)
dec_emb = dec_emb_layer(decoder_inputs)
dec_masking = Masking(mask_value=0)(dec_emb)

decoder_lstm = LSTM(hidden_units, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(dec_masking, initial_state=encoder_states)

decoder_dense = Dense(tar_vocab_size, activation="softmax")
decoder_outputs = decoder_dense(decoder_outputs)

In [23]:
#Model compile
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])

In [ ]:
#model training
"""
es = EarlyStopping(monitor="val_accuracy", mode="max", verbose=1, patience=4)
mc = ModelCheckpoint("Models/best_Seq2seq_model.h5", monitor="val_accuracy", mode="max", verbose=1, save_best_only=True)

model.fit(x=[encoder_input_train, decoder_input_train], y=decoder_target_train, validation_data=([encoder_input_test, decoder_input_test], decoder_target_test), batch_size=128, epochs=100, callbacks=[es, mc])
"""

model.fit(x=[encoder_input_train, decoder_input_train], y=decoder_target_train, validation_data=([encoder_input_test, decoder_input_test], decoder_target_test), batch_size=128, epochs=600)

Epoch 1/600
493/493 [==============================] - 43s 64ms/step - loss: 1.8081 - accuracy: 0.7431 - val_loss: 1.4656 - val_accuracy: 0.7746
Epoch 2/600
493/493 [==============================] - 28s 56ms/step - loss: 1.3305 - accuracy: 0.7909 - val_loss: 1.2418 - val_accuracy: 0.8064
Epoch 3/600
493/493 [==============================] - 28s 56ms/step - loss: 1.1028 - accuracy: 0.8176 - val_loss: 1.0759 - val_accuracy: 0.8251
Epoch 4/600
493/493 [==============================] - 28s 56ms/step - loss: 0.9223 - accuracy: 0.8387 - val_loss: 0.9542 - val_accuracy: 0.8427
Epoch 5/600
493/493 [==============================] - 27s 55ms/step - loss: 0.7753 - accuracy: 0.8570 - val_loss: 0.8594 - val_accuracy: 0.8559
Epoch 6/600
493/493 [==============================] - 27s 55ms/step - loss: 0.6493 - accuracy: 0.8738 - val_loss: 0.7863 - val_accuracy: 0.8677
Epoch 7/600
493/493 [==============================] - 27s 55ms/step - loss: 0.5389 - accuracy: 0.8899 - val_loss: 0.7286 - val_ac

493/493 [==============================] - 30s 60ms/step - loss: 0.0357 - accuracy: 0.9859 - val_loss: 0.8331 - val_accuracy: 0.8978
Epoch 58/600
493/493 [==============================] - 28s 57ms/step - loss: 0.0360 - accuracy: 0.9858 - val_loss: 0.8220 - val_accuracy: 0.8983
Epoch 59/600
493/493 [==============================] - 29s 58ms/step - loss: 0.0359 - accuracy: 0.9858 - val_loss: 0.8261 - val_accuracy: 0.8979
Epoch 60/600
493/493 [==============================] - 28s 57ms/step - loss: 0.0356 - accuracy: 0.9858 - val_loss: 0.8324 - val_accuracy: 0.8977
Epoch 61/600
493/493 [==============================] - 28s 57ms/step - loss: 0.0353 - accuracy: 0.9859 - val_loss: 0.8305 - val_accuracy: 0.8977
Epoch 62/600
493/493 [==============================] - 28s 57ms/step - loss: 0.0351 - accuracy: 0.9859 - val_loss: 0.8266 - val_accuracy: 0.8981
Epoch 63/600
493/493 [==============================] - 28s 56ms/step - loss: 0.0349 - accuracy: 0.9859 - val_loss: 0.8368 - val_accuracy

493/493 [==============================] - 27s 54ms/step - loss: 0.0295 - accuracy: 0.9860 - val_loss: 0.8880 - val_accuracy: 0.8969
Epoch 114/600
493/493 [==============================] - 27s 55ms/step - loss: 0.0296 - accuracy: 0.9860 - val_loss: 0.8825 - val_accuracy: 0.8967
Epoch 115/600
493/493 [==============================] - 27s 54ms/step - loss: 0.0294 - accuracy: 0.9861 - val_loss: 0.8895 - val_accuracy: 0.8967
Epoch 116/600
493/493 [==============================] - 27s 54ms/step - loss: 0.0294 - accuracy: 0.9860 - val_loss: 0.8896 - val_accuracy: 0.8971
Epoch 117/600
493/493 [==============================] - 27s 54ms/step - loss: 0.0292 - accuracy: 0.9860 - val_loss: 0.8933 - val_accuracy: 0.8967
Epoch 118/600
493/493 [==============================] - 27s 54ms/step - loss: 0.0292 - accuracy: 0.9861 - val_loss: 0.8887 - val_accuracy: 0.8962
Epoch 119/600
493/493 [==============================] - 27s 54ms/step - loss: 0.0293 - accuracy: 0.9860 - val_loss: 0.9004 - val_ac

Epoch 169/600
493/493 [==============================] - 27s 54ms/step - loss: 0.0267 - accuracy: 0.9860 - val_loss: 0.9308 - val_accuracy: 0.8954
Epoch 170/600
493/493 [==============================] - 27s 54ms/step - loss: 0.0268 - accuracy: 0.9861 - val_loss: 0.9443 - val_accuracy: 0.8953
Epoch 171/600
493/493 [==============================] - 27s 54ms/step - loss: 0.0272 - accuracy: 0.9860 - val_loss: 0.9428 - val_accuracy: 0.8955
Epoch 172/600
493/493 [==============================] - 27s 54ms/step - loss: 0.0272 - accuracy: 0.9859 - val_loss: 0.9428 - val_accuracy: 0.8950
Epoch 173/600
493/493 [==============================] - 26s 54ms/step - loss: 0.0271 - accuracy: 0.9860 - val_loss: 0.9425 - val_accuracy: 0.8953
Epoch 174/600
493/493 [==============================] - 26s 54ms/step - loss: 0.0268 - accuracy: 0.9861 - val_loss: 0.9515 - val_accuracy: 0.8955
Epoch 175/600
493/493 [==============================] - 26s 54ms/step - loss: 0.0265 - accuracy: 0.9860 - val_loss: 0

Epoch 225/600
493/493 [==============================] - 26s 54ms/step - loss: 0.0263 - accuracy: 0.9859 - val_loss: 1.0001 - val_accuracy: 0.8938
Epoch 226/600
493/493 [==============================] - 26s 54ms/step - loss: 0.0261 - accuracy: 0.9860 - val_loss: 0.9910 - val_accuracy: 0.8939
Epoch 227/600
493/493 [==============================] - 26s 54ms/step - loss: 0.0257 - accuracy: 0.9861 - val_loss: 0.9972 - val_accuracy: 0.8942
Epoch 228/600
493/493 [==============================] - 26s 54ms/step - loss: 0.0252 - accuracy: 0.9860 - val_loss: 1.0038 - val_accuracy: 0.8946
Epoch 229/600
493/493 [==============================] - 26s 54ms/step - loss: 0.0250 - accuracy: 0.9861 - val_loss: 0.9926 - val_accuracy: 0.8942
Epoch 230/600
493/493 [==============================] - 26s 54ms/step - loss: 0.0249 - accuracy: 0.9861 - val_loss: 1.0036 - val_accuracy: 0.8943
Epoch 231/600
493/493 [==============================] - 26s 54ms/step - loss: 0.0249 - accuracy: 0.9861 - val_loss: 1

Epoch 281/600
493/493 [==============================] - 26s 54ms/step - loss: 0.0255 - accuracy: 0.9860 - val_loss: 1.0324 - val_accuracy: 0.8925
Epoch 282/600
493/493 [==============================] - 26s 54ms/step - loss: 0.0257 - accuracy: 0.9859 - val_loss: 1.0363 - val_accuracy: 0.8926
Epoch 283/600
493/493 [==============================] - 26s 54ms/step - loss: 0.0255 - accuracy: 0.9859 - val_loss: 1.0217 - val_accuracy: 0.8917
Epoch 284/600
493/493 [==============================] - 26s 54ms/step - loss: 0.0249 - accuracy: 0.9860 - val_loss: 1.0374 - val_accuracy: 0.8928
Epoch 285/600
493/493 [==============================] - 26s 54ms/step - loss: 0.0242 - accuracy: 0.9861 - val_loss: 1.0287 - val_accuracy: 0.8929
Epoch 286/600
493/493 [==============================] - 26s 54ms/step - loss: 0.0238 - accuracy: 0.9862 - val_loss: 1.0344 - val_accuracy: 0.8934
Epoch 287/600
493/493 [==============================] - 26s 54ms/step - loss: 0.0238 - accuracy: 0.9862 - val_loss: 1

Epoch 337/600
493/493 [==============================] - 30s 61ms/step - loss: 0.0242 - accuracy: 0.9861 - val_loss: 1.0674 - val_accuracy: 0.8924
Epoch 338/600
493/493 [==============================] - 31s 63ms/step - loss: 0.0250 - accuracy: 0.9860 - val_loss: 1.0676 - val_accuracy: 0.8914
Epoch 339/600
493/493 [==============================] - 30s 60ms/step - loss: 0.0267 - accuracy: 0.9856 - val_loss: 1.0737 - val_accuracy: 0.8908
Epoch 340/600
493/493 [==============================] - 30s 60ms/step - loss: 0.0265 - accuracy: 0.9857 - val_loss: 1.0711 - val_accuracy: 0.8914
Epoch 341/600
493/493 [==============================] - 32s 64ms/step - loss: 0.0249 - accuracy: 0.9859 - val_loss: 1.0734 - val_accuracy: 0.8918
Epoch 342/600
493/493 [==============================] - 31s 63ms/step - loss: 0.0238 - accuracy: 0.9861 - val_loss: 1.0687 - val_accuracy: 0.8920
Epoch 343/600
493/493 [==============================] - 31s 62ms/step - loss: 0.0234 - accuracy: 0.9860 - val_loss: 1

In [ ]:
#model = load_model("Models/best_Seq2seq_model.h5")

# Activating Model

In [ ]:
#Encoder
encoder_model = Model(encoder_inputs, encoder_states)

In [ ]:
#Decoder
decoder_state_input_h = Input(shape=(hidden_units,))
decoder_state_input_c = Input(shape=(hidden_units,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

dec_emb2 = dec_emb_layer(decoder_inputs) #Shared layer

decoder_outputs2, state_h2, state_c2 = decoder_lstm(dec_emb2, initial_state=decoder_states_inputs)
decoder_states2 = [state_h2, state_c2]

decoder_outputs2 = decoder_dense(decoder_outputs2)

decoder_model = Model([decoder_inputs] + decoder_states_inputs, [decoder_outputs2] + decoder_states2)

In [ ]:
#Activating Decoder
def decode_sequence(input_seq) :
    states_value = encoder_model.predict(input_seq)
    
    target_seq = np.zeros((1,1))
    target_seq[0,0] = tar_to_index["<sos>"]
    
    decoded_sentence = ''
    stop_condition = True
    
    while stop_condition :
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)
        
        sampled_token_index = np.argmax(output_tokens[0, -1, :]) #가장 마지막 행 전체
        sampled_char = index_to_tar[sampled_token_index]
        
        decoded_sentence += ' ' + sampled_char
        
        if (sampled_char == '<eos>' or len(decoded_sentence) > 50) :
            stop_condition = False
        
        target_seq = np.zeros((1,1))
        target_seq[0,0] = sampled_token_index
        
        sates_value = [h, c]
        
    return decoded_sentence

#  Activate

In [ ]:
def seq_to_src(input_seq):
    sentence = ''
    for encoded_word in input_seq:
        if(encoded_word != 0):
            sentence = sentence + index_to_src[encoded_word] + ' '
            
    return sentence

def seq_to_tar(input_seq):
    sentence = ''
    for encoded_word in input_seq:
        if(encoded_word != 0 and encoded_word != tar_to_index['<sos>'] and encoded_word != tar_to_index['<eos>']):
            sentence = sentence + index_to_tar[encoded_word] + ' '
            
    return sentence

In [ ]:
for seq_index in [3, 100, 300, 500, 2001]:
    input_seq = encoder_input_train[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)

    print("Input sentence :",seq_to_src(encoder_input_train[seq_index]))
    print("Correct sentence :",seq_to_tar(decoder_input_train[seq_index]))
    print("Output sentence :",decoded_sentence)
    print("-"*70)